In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""Main script."""
from tensorflow.keras.callbacks import TensorBoard, CSVLogger
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import os
import model as mod
import dataset
from dataset import plot_piano_roll
from pathlib import Path
from plotter import smooth
import time
import math
import ev_metrics

# P = Path(__file__).parent.absolute()
P = Path(os.path.abspath(''))  # Compatible with Jupyter Notebook

FS = 10  # Sampling frequency. 10 Hz = 100 ms

In [ ]:
# Load midi files.
midi_list = [x for x in os.listdir(P / "data") if x.endswith('.mid')]
st = 10
num_ts = 10

# All dataset
train_list = midi_list[0:165]
validation_list = midi_list[166:213]
test_list = midi_list[213:236]

# Small dataset
# train_list = midi_list[0:20]
# validation_list = midi_list[20:30]
# test_list = midi_list[61:65]

print("Train list:  ", train_list)
print("Validation list:  ", validation_list)
print("Test list:  ", test_list)

In [ ]:
# Create generators.
BASELINE = 0
train = dataset.DataGenerator(train_list, P / "data",  fs=FS, bl=BASELINE)
validation = dataset.DataGenerator(validation_list, P / "data",  fs=FS, bl=BASELINE)
test = dataset.DataGenerator(test_list, P / "data",  fs=FS, bl=0)
train.build_dataset("training", step=st, t_step=num_ts)
validation.build_dataset("validation", step=st, t_step=num_ts)
test.build_dataset("test", step=st, t_step=num_ts)
print("Done")

In [ ]:
print(train.dataset[0].shape)
print(train.dataset[0][:,0,:].shape)
print(train.dataset[1][:,0:88].shape)

In [ ]:
# Data augmentation
# train.build_transposed()

In [ ]:
 # Build Keras model.
model = mod.build_model((st, 88), num_ts)
model.summary()
now = datetime.now()

# Save logs
logger = TensorBoard(log_dir=P / 'logs' / now.strftime("%Y%m%d-%H%M%S"),
                     write_graph=True, update_freq='epoch')

csv_logger = CSVLogger(P / 'logs' / (now.strftime("%Y%m%d-%H%M%S") + '-' +
                       str(st) + '-' + str(num_ts) + '.csv'),
                       separator=',', append=False)

In [ ]:
# Fit the model.

BS = 64  # Batch size
epochs = 10
start = time.time()
size_train = math.ceil(train.dataset[0].shape[0] / BS)
spe_train = size_train #+ size_train*10
size_valid = math.ceil(validation.dataset[0].shape[0] / BS)
spe_valid = size_valid #+ size_valid*10
print("Train dataset shape: ", train.dataset[0].shape, "\n")

# Fit generator. Data should be shuffled before fitting.
# history = model.fit(train.generate(bs=BS, limit=epochs, trans=0, name='train'), epochs=epochs,
#           steps_per_epoch=spe_train,
#           validation_data=validation.generate(bs=BS, limit=epochs, trans=0, name='valid'),
#           validation_steps=spe_valid,
#           shuffle=True,
#           callbacks=[logger, csv_logger])


# Normal fit. Auto-shuffles data.
history = model.fit(x=train.dataset[0], y=train.dataset[1],
                    epochs=epochs, batch_size=BS, shuffle=True,
                    validation_data=(validation.dataset[0],
                                     validation.dataset[1]),
                    callbacks=[logger, csv_logger])

end = time.time()

In [ ]:
hist = pd.DataFrame(history.history)
print(hist)

In [ ]:
plt.figure(constrained_layout=True, figsize=(5, 4))
# plt.plot(smooth(hist['val_f1_first']), label='First')
# plt.plot(smooth(hist['val_f1_second']), label='Second')
# plt.plot(smooth(hist['val_f1_last']), label='Last')

# plt.plot(smooth(hist['val_f1_2']), label='2')
# plt.plot(smooth(hist['val_f1_3']), label='3')
# plt.plot(smooth(hist['val_f1_4']), label='4')
# plt.plot(smooth(hist['val_f1_5']), label='5')
# plt.plot(smooth(hist['val_f1_6']), label='6')
# plt.plot(smooth(hist['val_f1_7']), label='7')
# plt.plot(smooth(hist['val_f1_8']), label='8')

# plt.plot(smooth(hist['val_f1_first']), label='Validation')
# plt.plot(smooth(hist['f1_first']), label='Train')

plt.plot(smooth(hist['val_loss']), 'x-', c='tab:orange', label='Validation', ms=8)
plt.plot(smooth(hist['loss']), 'ro-', c='tab:red', label='Train')
plt.xlabel('Epoch')
plt.xticks(range(epochs))
plt.legend()
plt.title('Loss: Binary cross-entropy')
plt.savefig('loss.eps', format='eps')
plt.show()

In [ ]:
# Evaluate the model.
print("Evaluation on train set:")
e_train = model.evaluate(x=train.dataset[0],
                         y=train.dataset[1],
                         batch_size=BS)

print("\nEvaluation on validation set:")
e_valid = model.evaluate(x=validation.dataset[0],
                         y=validation.dataset[1],
                         batch_size=BS)

print("\nEvaluation on test set:")
e_test = model.evaluate(x=test.dataset[0],
                        y=test.dataset[1],
                        batch_size=BS)

results = {out: e_train[i] for i, out in enumerate(model.metrics_names)}
res = pd.DataFrame(list(results.items()), columns=['metric', 'train'])
res = res.set_index('metric')

results2 = {out: e_valid[i] for i, out in enumerate(model.metrics_names)}
res2 = pd.DataFrame(list(results2.items()), columns=['metric', 'validation'])
res2 = res2.set_index('metric')

results3 = {out: e_test[i] for i, out in enumerate(model.metrics_names)}
res3 = pd.DataFrame(list(results3.items()), columns=['metric', 'test'])
res3 = res3.set_index('metric')


result = pd.concat([res, res2, res3], axis=1, sort=False)
result

In [ ]:
# test = dataset.DataGenerator(test_list, P / "data",  fs=FS, bl=0)
# test.build_dataset("test", step=st, t_step=num_ts)

predictions = model.predict(x=test.dataset[0])
print("Pred shape: ", predictions.shape)
predictions = predictions[:, 88*0:88*1]
print(predictions.shape)
print(test.dataset[0].shape, "\n\n\n")
test2 = test.dataset[0][:, 0, :]
predictions2 = dataset.transpose(predictions)
predictions2 = dataset.convert(predictions2)
# test = test[:, 0, :]
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
plot_piano_roll(dataset.transpose(test2), 21, 109, ax1, FS)
ax1.set_title('Test')

plot_piano_roll(predictions2, 21, 109, ax2, FS)
ax2.set_title('Predictions')
plt.show()
print("Training time: ", (end-start))

In [ ]:
import importlib
importlib.reload(ev_metrics)

predictions = model.predict(x=test.dataset[0])
baseline = dataset.DataGenerator(test_list, P / "data",  fs=FS, bl=1)
baseline.build_dataset("test", step=st, t_step=num_ts)
baseline.dataset[0].shape

print("")
print("Baseline shape: ", baseline.dataset[1].shape)
print("Test shape: ", test.dataset[1].shape)
print("Prediction shape: ", predictions.shape)
print("--- --- ---")
pred_auc, pred_f1 = ev_metrics.compute_auc(test.dataset[1], dataset.convert(predictions))
base_auc, base_f1 = ev_metrics.compute_auc(test.dataset[1], baseline.dataset[1])
print("Predictions mean AUC: ", pred_auc)
print("Baseline mean AUC: ", base_auc)

In [ ]:
plt.figure(constrained_layout=True, figsize=(5, 4))
plt.plot(range(1, num_ts + 1), pred_auc, 'x-', c='tab:blue', label='prediction', ms=8)
plt.plot(range(1, num_ts + 1), base_auc, 'o-', c='tab:green', label='baseline')
plt.legend()
plt.title('AUC on single timestep')
plt.xlabel('Timestep')
plt.xticks([0, 2, 4, 6, 8, 10])
plt.ylabel('ROC AUC')
name = 'auc' + str()
plt.savefig('auc.eps', format='eps')
plt.show()

In [ ]:
plt.figure(constrained_layout=True, figsize=(5, 4))
plt.plot(range(1, num_ts + 1), pred_f1, 'x-', c='tab:blue', label='prediction', ms=8)
plt.plot(range(1, num_ts + 1), base_f1, 'o-', c='tab:green', label='baseline')
plt.legend()
plt.title('F1 on single timestep')
plt.xlabel('Timestep')
plt.xticks([0, 2, 4, 6, 8, 10])
plt.ylabel('F1')
name = 'auc' + str()
plt.savefig('fscore.eps', format='eps')
plt.show()

In [ ]:
print("Predictions mean AUC: ", pred_auc)
print("Baseline mean AUC: ", base_auc)

In [ ]:
print("Predictions mean AUC: ", pred_f1)
print("Baseline mean AUC: ", base_f1)

In [ ]:
print("Pred shape: ", predictions.shape)
predictions_ = predictions[:, 88*0:88*1]
print(predictions_.shape)
print(test.dataset[0].shape, "\n\n\n")
test2 = test.dataset[0][:, 0, :]
predictions2 = dataset.transpose(predictions_)
predictions2 = dataset.convert(predictions2)
# test = test[:, 0, :]
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
plot_piano_roll(dataset.transpose(test2), 21, 109, ax1, FS)
ax1.set_title('Test')

plot_piano_roll(predictions2, 21, 109, ax2, FS)
ax2.set_title('Predictions')
plt.show()
print("Training time: ", (end-start))

In [ ]:
test = dataset.DataGenerator(test_list, P / "data",  fs=FS, bl=0)
test.build_dataset("test", step=st, t_step=num_ts)
baseline = dataset.DataGenerator(test_list, P / "data",  fs=FS, bl=1)
baseline.build_dataset("test", step=st, t_step=num_ts)
baseline.dataset[0].shape
base_auc, base_f1 = ev_metrics.compute_auc(test.dataset[1], baseline.dataset[1])


In [ ]:
plt.figure(constrained_layout=True, figsize=(5, 4))
# plt.plot(range(1, num_ts + 1), pred_auc, 'x-', c='tab:blue', label='prediction', ms=8)
plt.plot(range(1, num_ts + 1), base_auc, 'o-', c='tab:green', label='baseline')
plt.legend()
plt.title('AUC on single timestep')
plt.xlabel('Timestep')
# plt.xticks([0, 2, 4, 6, 8, 10,])
plt.ylabel('ROC AUC')
name = 'auc' + str()
plt.savefig('auc.eps', format='eps')
plt.show()